# 피봇테이블

In [1]:
import pandas as pd
import numpy as np

피봇테이블(pivot table)이란 데이터 열 중에서 두 개의 열을 각각 행 인덱스, 열 인덱스로 사용하여 데이터를 조회하여 펼쳐놓은 것을 말한다.

Pandas는 피봇테이블을 만들기 위한 pivot 메서드를 제공한다. 첫번째 인수로는 행 인덱스로 사용할 열 이름, 두번째 인수로는 열 인덱스로 사용할 열 이름, 그리고 마지막으로 데이터로 사용할 열 이름을 넣는다.

Pandas는 지정된 두 열을 각각 행 인덱스와 열 인덱스로 바꾼 후 행 인덱스의 라벨 값이 첫번째 키의 값과 같고 열 인덱스의 라벨 값이 두번째 키의 값과 같은 데이터를 찾아서 해당 칸에 넣는다. 만약 주어진 데이터가 존재하지 않으면 해당 칸에 NaN 값을 넣는다.

다음 데이터는 각 도시의 연도별 인구를 나타낸 것이다.

In [2]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


어떤 도시의 어떤 시점의 인구를 쉽게 알 수 있도록 피봇테이블로 만들어보자. pivot 명령으로 사용하고 행 인덱스 인수로는 "도시", 열 인덱스 인수로는 "연도", 데이터 이름으로 "인구"를 입력하면 된다.

In [3]:
df1.pivot("도시", "연도", "인구")

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


사실 피봇테이블은 다음과 같이 set_index 명령과 unstack 명령을 사용해서 만들 수도 있다.

In [4]:
df1.set_index(["도시", "연도"])[["인구"]].unstack()

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

키 값으로 데이터가 단 하나만 찾아져야 한다. 만약 행 인덱스와 열 인덱스 조건을 만족하는 데이터가 2개 이상인 경우에는 에러가 발생한다. 예를 들어 위 데이터프레임에서 ("지역", "연도")를 키로 하면 ("수도권", "2015")에 해당하는 값이 두 개 이상이므로 다음과 같이 에러가 발생한다.

In [5]:
df1.pivot("지역", "연도", "인구")

ValueError: Index contains duplicate entries, cannot reshape

### 그룹분석
그룹분석은 피봇테이블과 달리 키에 의해서 결정되는 데이터가 여러개가 있을 경우 미리 지정한 연산을 통해 그 그룹 데이터의 대표값을 계산한. Pandas에서는 groupby 명령을 사용하여 다음처럼 그룹분석을 한다.

* 분석하고자 하는 시리즈나 데이터프레임에 groupby 메서드를 호출하여 그룹화를 한다.
* 그룹 객체에 대해 그룹연산을 수행한다.

groupby 결과, 즉 GroupBy 클래스 객체의 뒤에 붙일 수 있는 그룹연산 메서드는 다양하다. 다음은 자주 사용되는 그룹연산 메서드들이다.

* size, count: 그룹 데이터의 갯수
* mean, median, min, max: 그룹 데이터의 평균, 중앙값, 최소, 최대
* sum, prod, std, var, quantile : 그룹 데이터의 합계, 곱, 표준편차, 분산, 사분위수
* first, last: 그룹 데이터 중 가장 첫번째 데이터와 가장 나중 데이터

In [6]:
# 예를 들어 다음과 같은 데이터가 있을 때 key1의 값(A 또는 B)에 따른 data1의 평균은 어떻게 구할까?
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


groupby 명령을 사용하여 그룹 A와 그룹 B로 구분한 그룹 데이터를 만든다.

In [7]:
groups = df2.groupby(df2.key1)
groups

이 GroupBy 클래스 객체에는 각 그룹 데이터의 인덱스를 저장한 groups 속성이 있다.

In [8]:
groups.groups

{'A': Int64Index([0, 1, 4], dtype='int64'),
 'B': Int64Index([2, 3], dtype='int64')}

In [9]:
# A그룹과 B그룹 데이터의 합계를 구하기 위해 sum이라는 그룹연산을 한다.
groups.sum()

,data1,data2
key1,,
A,8,80
B,7,70


데이터를 그룹으로 나눈 GroupBy 클래스 객체 또는 그룹분석한 결과에서 data1만 뽑아도 된다.

In [10]:
df2.groupby(df2.key1)["data1"].sum()

key1
A    8
B    7
Name: data1, dtype: int64

### 연습 문제 1
key1의 값을 기준으로 data1의 값을 분류하여 합계를 구한 결과를 시리즈가 아닌 데이터프레임으로 구한다.

In [11]:
# TODO

복합 키 (key1, key2) 값에 따른 data1의 합계를 구하자. 분석하고자 하는 키가 복수이면 리스트를 사용한다.

In [12]:
df2.data1.groupby([df2.key1, df2.key2]).sum()

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

이 결과를 unstack 명령으로 피봇 데이블 형태로 만들수도 있다.

In [13]:
df2.data1.groupby([df2["key1"], df2["key2"]]).sum().unstack("key2")

key2,one,two
key1,,
A,6,2
B,3,4


그룹분석 기능을 사용하면 위의 인구 데이터로부터 지역별 합계를 구할 수도 있다.

In [14]:
df1["인구"].groupby([df1["지역"], df1["연도"]]).sum().unstack("연도")

연도,2005,2010,2015
지역,,,
경상권,3512547,3393191,3448737
수도권,9762546,9894685,12794763


다음과 같은 그룹연산이 있다.

* agg, aggregate

만약 원하는 그룹연산이 없는 경우 함수를 만들고 이 함수를 agg에 전달한다.
또는 여러가지 그룹연산을 동시에 하고 싶은 경우 함수 이름 문자열의 리스트를 전달한다.
* describe

하나의 그룹 대표값이 아니라 여러개의 값을 데이터프레임으로 구한다.
* apply

describe 처럼 하나의 대표값이 아닌 데이터프레임을 출력하지만 원하는 그룹연산이 없는 경우에 사용한다.
* transform

그룹에 대한 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터 자체를 변형한다.

In [15]:
# iris dataset load
# Data Loading
iris = pd.read_csv("../data/iris.csv")

각 붓꽃 종별로 가장 큰 값과 가장 작은 값의 비율을 구해보자. 이러한 계산을 하는 그룹연산 메서드는 없으므로 직접 만든 후 agg 메서드를 적용한다.

In [16]:
def peak_to_peak_ratio(x):
    return x.max() / x.min()

iris.groupby(iris.species).agg(peak_to_peak_ratio)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


describe 메서드를 사용하면 다양한 기술 통계(descriptive statistics)값을 한 번에 구한다.

In [17]:
iris.groupby(iris.species).describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000

apply 메서드를 사용하면 describe 메서드처럼 하나의 그룹에 대해 하나의 대표값(스칼라 값)을 구하는 게 아니라 데이터프레임을 만들 수 있다. 예를 들어 다음처럼 각 붓꽃 종별로 가장 꽃잎 길이(petal length)가 큰 3개의 데이터를 뽑아낼 수도 있다.

In [18]:
def top3_petal_length(df):
    return df.sort_values(by="petal_length", ascending=False)[:3]

iris.groupby(iris.species).apply(top3_petal_length)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

transform 메서드는 그룹별 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터프레임 자체를 변화시킨다. 따라서 만들어진 데이터프레임의 크기는 원래 데이터프레임과 같다. 예를 들어 다음처럼 각 붓꽃이 해당 종 중에서 꽃잎길이가 대/중/소 어느 것에 해당되는지에 대한 데이터프레임을 만들 수도 있다.

In [19]:
def q3cut(s):
    return pd.qcut(s, 3, labels=["소", "중", "대"])


iris["petal_length_class"] = iris.groupby(iris.species)["petal_length"].transform(q3cut)
iris[["petal_length", "petal_length_class"]].tail(10)

,petal_length,petal_length_class
140,5.6,중
141,5.1,소
142,5.1,소
143,5.9,대
144,5.7,중
145,5.2,소
146,5.0,소
147,5.2,소
148,5.4,중
149,5.1,소


### 연습 문제 2
붓꽃(iris) 데이터에서 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width) 등의 평균을 구하라. 만약 붓꽃 종(species)이 표시되지 않았을 때 이 수치들을 이용하여 붓꽃 종을 찾아낼 수 있을지 생각하라.

In [20]:
# TODO

### pivot_table
Pandas는 pivot 명령과 groupby 명령의 중간 성격을 가지는 pivot_table 명령도 제공한다.

pivot_table 명령은 groupby 명령처럼 그룹분석을 하지만 최종적으로는 pivot 명령처럼 피봇테이블을 만든다. 즉 groupby 명령의 결과에 unstack을 자동 적용하여 2차원적인 형태로 변형한다. 사용 방법은 다음과 같다.

```
pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, margins_name='All')
```
* data: 분석할 데이터프레임 (메서드일 때는 필요하지 않음)
* values: 분석할 데이터프레임에서 분석할 열
* index: 행 인덱스로 들어갈 키 열 또는 키 열의 리스트
* columns: 열 인덱스로 들어갈 키 열 또는 키 열의 리스트
* aggfunc: 분석 메서드
* fill_value: NaN 대체 값
* margins: 모든 데이터를 분석한 결과를 오른쪽과 아래에 붙일지 여부
* margins_name: 마진 열(행)의 이름


만약 조건에 따른 데이터가 유일하게 선택되지 않으면 그룹연산을 하며 이 때 aggfunc 인수로 정의된 함수를 수행하여 대표값을 계산한다.

In [21]:
df1.pivot_table("인구", "도시", "연도")

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


margins=True 인수를 주면 aggfunc로 주어진 분석 방법을 해당 열의 모든 데이터, 해당 행의 모든 데이터 그리고 전체 데이터에 대해 적용한 결과를 같이 보여준다. aggfunc가 주어지지 않았으면 평균을 계산한다.

In [22]:
df1.pivot_table("인구", "도시", "연도", margins=True, margins_name="합계")

연도,2005,2010,2015,합계
도시,,,,
부산,3512547.0,3393191.0,3448737.0,3.451492e+06
서울,9762546.0,9631482.0,9904312.0,9.766113e+06
인천,NaN,263203.0,2890451.0,1.576827e+06
합계,6637546.5,4429292.0,5414500.0,5.350809e+06


행 인덱스나 열 인덱스에 리스트를 넣으면 다중 인덱스 테이블을 만든다.

In [23]:
df1.pivot_table("인구", index=["연도", "도시"])

인구
연도   도시         
2005 부산  3512547
     서울  9762546
2010 부산  3393191
     서울  9631482
     인천   263203
2015 부산  3448737
     서울  9904312
     인천  2890451

### TIP 데이터 예제
식당에서 식사 후 내는 팁(tip)과 관련된 데이터를 이용하여 좀더 구체적으로 그룹분석 방법을 살펴본다. 우선 Seaborn 패키지에 설치된 샘플 데이터를 로드한다. 이 데이터프레임에서 각각의 컬럼은 다음을 뜻한다.

* total_bill: 식사대금
* tip: 팁
* sex: 성별
* smoker: 흡연/금연 여부
* day: 요일
* time: 시간
* size: 인원

In [24]:
tips = pd.read_csv("../data/tips.csv")
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


식사대금와 팁의 비율을 나타내는 tip_pct를 추가하자.

In [25]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204
243,18.78,3.00,Female,No,Thur,Dinner,2,0.159744


In [26]:
tips.describe()

,total_bill,tip,size,tip_pct
count,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,0.160803
std,8.902412,1.383638,0.951100,0.061072
min,3.070000,1.000000,1.000000,0.035638
25%,13.347500,2.000000,2.000000,0.129127
50%,17.795000,2.900000,2.000000,0.154770
75%,24.127500,3.562500,3.000000,0.191475
max,50.810000,10.000000,6.000000,0.710345


우선 성별로 나누어 데이터 갯수를 세어본다.

In [27]:
tips.groupby("sex").count()

,total_bill,tip,smoker,day,time,size,tip_pct
sex,,,,,,,
Female,87,87,87,87,87,87,87
Male,157,157,157,157,157,157,157


성별과 흡연유무로 나누어 데이터의 갯수를 알아본다.

In [28]:
tips.groupby(["sex", "smoker"]).size()

sex     smoker
Female  No        54
        Yes       33
Male    No        97
        Yes       60
dtype: int64

피봇 데이블 형태로 바꿀 수도 있다.

In [29]:
tips.pivot_table("tip_pct", "sex", "smoker", aggfunc="count", margins=True)

smoker,No,Yes,All
sex,,,
Female,54,33,87
Male,97,60,157
All,151,93,244


성별과 흡연 여부에 따른 평균 팁 비율을 살펴본다.

In [30]:
tips.groupby("sex")[["tip_pct"]].mean()

,tip_pct
sex,
Female,0.166491
Male,0.157651


In [31]:
tips.groupby("smoker")[["tip_pct"]].mean()

,tip_pct
smoker,
No,0.159328
Yes,0.163196


In [32]:
tips.pivot_table("tip_pct", "sex")

,tip_pct
sex,
Female,0.166491
Male,0.157651


In [33]:
tips.pivot_table("tip_pct", ["sex", "smoker"])

tip_pct
sex    smoker          
Female No      0.156921
       Yes     0.182150
Male   No      0.160669
       Yes     0.152771

In [34]:
tips.pivot_table("tip_pct", "sex", "smoker")

smoker,No,Yes
sex,,
Female,0.156921,0.182150
Male,0.160669,0.152771


다른 통계값이 없으므로 describe 명령으로 여러가지 통계값을 한 번에 알아본다.

In [35]:
tips.groupby("sex")[["tip_pct"]].describe()

tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
sex                                                                          
Female    87.0  0.166491  0.053632  0.056433  0.140416  0.155581  0.194266   
Male     157.0  0.157651  0.064778  0.035638  0.121389  0.153492  0.186240   

                  
             max  
sex               
Female  0.416667  
Male    0.710345

In [36]:
tips.groupby("smoker")[["tip_pct"]].describe()

tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
smoker                                                                       
No       151.0  0.159328  0.039910  0.056797  0.136906  0.155625  0.185014   
Yes       93.0  0.163196  0.085119  0.035638  0.106771  0.153846  0.195059   

                  
             max  
smoker            
No      0.291990  
Yes     0.710345

In [37]:
tips.groupby(["sex", "smoker"])[["tip_pct"]].describe()

tip_pct                                                    \
                count      mean       std       min       25%       50%   
sex    smoker                                                             
Female No        54.0  0.156921  0.036421  0.056797  0.139708  0.149691   
       Yes       33.0  0.182150  0.071595  0.056433  0.152439  0.173913   
Male   No        97.0  0.160669  0.041849  0.071804  0.131810  0.157604   
       Yes       60.0  0.152771  0.090588  0.035638  0.101845  0.141015   

                                   
                    75%       max  
sex    smoker                      
Female No      0.181630  0.252672  
       Yes     0.198216  0.416667  
Male   No      0.186220  0.291990  
       Yes     0.191697  0.710345

### 연습 문제 3
1. 팁의 비율이 요일과 점심/저녁 여부, 인원수에 어떤 영향을 받는지 살펴본다.
2. 어떤 요인이 가장 크게 작용하는지 판단할 수 있는 방법이 있는가?

In [38]:
# TODO